## Packages

In [1]:
import pandas as pd
import re
import string
import nltk
from sklearn.manifold import TSNE
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

## Functions

In [2]:
def transformDocument(df, column_name, language):
    stop_words = usingStopwords(language)
    df_pp = df.copy()
    # 1. Aplicar preprocessamento nos títulos e textos completos
    if language == 'pt':
        # Substituir símbolos importantes
        df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('-feira', ''))
        df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('+', 'mais '))
        df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('-', 'menos '))
        df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('%', ' por cento'))
        df_pp[column_name] = df_pp[column_name].map(lambda s: removeStopwords(s, stop_words))

    elif language == 'en':
        df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('-', 'less'))
        df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('+', 'plus '))
        df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('%', ' percent'))
        df_pp[column_name] = df_pp[column_name].map(lambda s: removeStopwords(s, stop_words))

    else:
        pass

    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('R$', ''))
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('U$', ''))
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('US$', ''))
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('S&P 500', 'spx'))

    # Transformar em String e Letras Minúsculas nas Mensagens
    df_pp[column_name] = df_pp[column_name].map(lambda s:
                                              normalizarString(s))


    # Remover Pontuações
    # Remover Pontuações
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.translate(str.maketrans('', '', string.punctuation)))

    # Remover Emojis
    df_pp[column_name] = df_pp[column_name].map(lambda s: removeEmojis(s))

    # Quebras de Linha desnecessárias
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('\n', ' '))

    # Remover aspas duplas
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('\"', ''))
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('“', ''))
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.replace('”', ''))

    # Remover valores
    df_pp[column_name] = df_pp[column_name].map(lambda s: removeValores(s))

    # Espaços desnecessários
    df_pp[column_name] = df_pp[column_name].map(lambda s: s.strip())
    return df_pp



def removeEmojis(sentence):
    "Remoção de Emojis nas mensagens de texto."

    # Padrões dos Emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u'\U00010000-\U0010ffff'
                               u"\u200d"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               u"\ufe0f"
                               "]+", flags=re.UNICODE)

    return emoji_pattern.sub(r'', sentence)

def removeValores(sentence):
    new_sentece = ''

    for token in sentence.split():
        if token.isdigit():
            token = '<NUM>'
        new_sentece += ' {}'.format(token)

    return new_sentece

def usingStopwords(language):
    stop_words = []

    nltk.download('stopwords')

    if language == 'pt':
        stop_words = nltk.corpus.stopwords.words('portuguese')
    elif language == 'en':
        stop_words = nltk.corpus.stopwords.words('english')

    return stop_words

def removeStopwords(text, stop_words):
    tokens = []
    for word in text.split():
        if word not in stop_words:
            tokens.append(word)

    text = ' '.join(tokens)
    return text

def normalizarString(text):
    """
    Função para retirar acentuações e converter para minúscula
    :param text:
    :return text_normalizado
    """
    import unicodedata

    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")
    return str(text.lower())

## Dependencies

In [3]:
train_raw = pd.read_csv('../assets/data/splits/train/raw.csv')
val_raw = pd.read_csv('../assets/data/splits/val/raw.csv')
test_raw = pd.read_csv('../assets/data/splits/test/raw.csv')
df = pd.read_csv('../assets/data/df.csv')
df

,title,tags,link,label
0,Óleo que vazou foi extraído de três campos na ...,"['Agência Brasil', 'Meio Ambiente', 'Nordeste'...",https://www.moneytimes.com.br/oleo-que-vazou-f...,1
1,"Futuros do minério de ferro recuam 1,56% na bo...","['Mineração', 'Minério de ferro', 'Vale']",https://www.moneytimes.com.br/futuros-do-miner...,2
2,Conheça a Top Pick de 21 analistas para janeiro,"['Carteira Recomendada', 'Comprar ou vender?',...",https://www.moneytimes.com.br/conheca-a-top-pi...,1
3,Reino Unido deve aprovar vacina da Pfizer em p...,"['Pfizer', 'Reino Unido', 'vacina', 'vacina da...",https://www.sunoresearch.com.br/noticias/reino...,0
4,O que você ainda não leu sobre o reajuste feit...,"['Ações', 'Artigo', 'Biodiesel', 'Coluna do Pe...",https://www.moneytimes.com.br/o-que-voce-ainda...,1
...,...,...,...,...
18007,"Açúcar bruto sobe na ICE, café arábica perde f...","['Açúcar', 'Agro Times', 'Agronegócio', 'Brasi...",https://www.moneytimes.com.br/acucar-bruto-sob...,1
18008,"Fechamento: Ibovespa fica estável em 0,01%","['Ações', 'BM&FBovespa', 'CCR', 'CSN', 'Ecorod...",https://www.moneytimes.com.br/fechamento-ibove...,1
18009,Fed alerta sobre risco para economia e adota m...,"['EUA', 'Fed', 'federal reserve', 'Federal Res...",https://www.sunoresearch.com.br/noticias/fed-a...,0
18010,Escritórios dos EUA pretendem abrir ação colet...,"['Estados Unidos (EUA)', 'Vale (VALE3)']",https://www.suno.com.br/noticias/escritorios-e...,2


## Preprocessing splits

In [4]:
train_pp = transformDocument(train_raw, 'title', 'pt')
val_pp = transformDocument(val_raw, 'title', 'pt')
test_pp = transformDocument(test_raw, 'title', 'pt')
df_pp = transformDocument(df, 'title', 'pt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df_pp

,title,tags,link,label
0,oleo vazou extraido tres campos venezuela diz ...,"['Agência Brasil', 'Meio Ambiente', 'Nordeste'...",https://www.moneytimes.com.br/oleo-que-vazou-f...,1
1,futuros minerio ferro recuam <NUM> cento bolsa...,"['Mineração', 'Minério de ferro', 'Vale']",https://www.moneytimes.com.br/futuros-do-miner...,2
2,conheca top pick <NUM> analistas janeiro,"['Carteira Recomendada', 'Comprar ou vender?',...",https://www.moneytimes.com.br/conheca-a-top-pi...,1
3,reino unido deve aprovar vacina pfizer poucos ...,"['Pfizer', 'Reino Unido', 'vacina', 'vacina da...",https://www.sunoresearch.com.br/noticias/reino...,0
4,o ainda leu sobre reajuste feito petrobras e b...,"['Ações', 'Artigo', 'Biodiesel', 'Coluna do Pe...",https://www.moneytimes.com.br/o-que-voce-ainda...,1
...,...,...,...,...
18007,acucar bruto sobe ice cafe arabica perde forca,"['Açúcar', 'Agro Times', 'Agronegócio', 'Brasi...",https://www.moneytimes.com.br/acucar-bruto-sob...,1
18008,fechamento ibovespa fica estavel <NUM> cento,"['Ações', 'BM&FBovespa', 'CCR', 'CSN', 'Ecorod...",https://www.moneytimes.com.br/fechamento-ibove...,1
18009,fed alerta sobre risco economia adota media in...,"['EUA', 'Fed', 'federal reserve', 'Federal Res...",https://www.sunoresearch.com.br/noticias/fed-a...,0
18010,escritorios eua pretendem abrir acao coletiva ...,"['Estados Unidos (EUA)', 'Vale (VALE3)']",https://www.suno.com.br/noticias/escritorios-e...,2


## Outputs for EDA, word2vec and preprocessing

In [6]:
train_pp.to_csv('../assets/data/splits/train/preprocessed.csv', index=False)
val_pp.to_csv('../assets/data/splits/val/preprocessed.csv', index=False)
test_pp.to_csv('../assets/data/splits/test/preprocessed.csv', index=False)
df_pp.to_csv('../assets/data/df_pp.csv', index=False)